In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


Sales_train csv dosyasını date formatına göre oku

In [ ]:
sales = pd.read_csv("../input/competitive-data-science-predict-future-sales/sales_train.csv", parse_dates=['date'], infer_datetime_format=True,dayfirst=True) 

In [ ]:
sales.head()

In [ ]:
test = pd.read_csv("../input/competitive-data-science-predict-future-sales/test.csv")

In [ ]:
test.head()

# 1. Data Preparation
Drop the duplicate values

In [ ]:
#Sales subset
sales_column = ['date_block_num','shop_id','item_id','item_price','item_cnt_day']

In [ ]:
sales.drop_duplicates(sales_column, keep='first', inplace=True)

In [ ]:
sales.reset_index(drop=True, inplace=True)

For test.csv

In [ ]:
test_column = ['shop_id','item_id']

In [ ]:
test.drop_duplicates(test_column, keep='first', inplace=True)

In [ ]:
test.reset_index(drop=True, inplace=True)

if item_price < 0 than equal to Zero

In [ ]:
sales.loc[sales.item_price < 0,'item_price'] = 0

Normalization for item_cnt_day. The values of item_cnt_day compress from 0 to 1000

In [ ]:
sales['item_cnt_day'] = sales['item_cnt_day'].clip(0,1000)

In [ ]:
sales['item_price'].max()

In [ ]:
sales['item_price'] = sales['item_price'].clip(0,300000)

Sales veri seti için date formatına göre gruplandırıyoruz

In [ ]:
df = sales.groupby([sales.date.apply(lambda x: x.strftime('%Y-%m')), 'item_id', 'shop_id']).sum().reset_index()

In [ ]:
df.columns

In [ ]:
df = df[['date', 'item_id', 'shop_id', 'item_cnt_day']]

In [ ]:
#Değerler - cnt_day , kolon - date , indexleri ise item/shop id
df = df.pivot_table(index=['item_id', 'shop_id'], columns='date', values='item_cnt_day', fill_value=0).reset_index()


In [ ]:
df

Test veri seti ile df DataFrame ' i , test kesişimine göre birleştir birleştir

In [ ]:
data = pd.merge(test, df, on=['item_id', 'shop_id'], how='left').fillna(0)

In [ ]:
#Bağımlı değişken
y = y = data.iloc[:,-1:]
#Bağımsız değişken
x = data.iloc[:,3:]
x.drop(['2015-10'], axis = 1, inplace = True)

In [ ]:
y

In [ ]:
x

In [ ]:
#Decision Tree Regression için hazırlık
x = x.values
y = y.values.reshape(-1,1)

# 2. Model Train 

In [ ]:
from sklearn.tree import DecisionTreeRegressor

tree = DecisionTreeRegressor()

tree.fit(x,y)

y_head = tree.predict(x)

y_head = y_head.reshape(-1,1)

# 3. Root Mean Square Error

In [ ]:
#Root Mean Square Error
def rmse(predict,real):
    return np.sqrt(np.mean((predict-real) ** 2))

rmse(y_head,y)

# 4. Output

In [ ]:
d = data.loc[:,["ID"]]
d["item_cnt_month"] = y_head

In [ ]:
submission = pd.DataFrame(data=d)
submission.to_csv('submission.csv', index=False)
submission.head()